# QC1: DropletUtils

2025-07-22


## Remove empty droplets

Many droplets are empty, mainly consisted of ambient RNA.

We have to filter out these droplets.


1. Construct an ambient pool by summing up UMIs from low-count droplet
2. Test the possibility that ambient RNA accidentally generated this droplet profile
These process are wrapped up by package DropletUtils

In [1]:
library(Seurat)
library(scater)
library(DropletUtils)
library(scran)
library(ggplot2)

set.seed(42) # for reproducibility

rdatadir = "/BiO/data/QC/"
save_path = "QC1"

if (!dir.exists(save_path)) {
  dir.create(save_path)
}


Warning message:
“package ‘Seurat’ was built under R version 4.3.2”
Loading required package: SeuratObject

Loading required package: sp

Warning message:
“package ‘sp’ was built under R version 4.3.3”

Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Warning message:
“package ‘scater’ was built under R version 4.3.2”
Loading required package: SingleCellExperiment

Warning message:
“package ‘SingleCellExperiment’ was built under R version 4.3.2”
Loading required package: SummarizedExperiment

Warning message:
“package ‘SummarizedExperiment’ was built under R version 4.3.2”
Loading required package: MatrixGenerics

Warning message:
“package ‘MatrixGenerics’ was built under R version 4.3.3”
Loading required package: matrixStats

Warning message:
“package ‘matrixStats’ was built under R version 4.3.3”

Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, 

### Load data

In [2]:
rawsce_1 <- read10xCounts(paste0(rdatadir, "20094_0001_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)
rawsce_2 <- read10xCounts(paste0(rdatadir, "20094_0002_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)
rawsce_3 <- read10xCounts(paste0(rdatadir, "20094_0003_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)
rawsce_4 <- read10xCounts(paste0(rdatadir, "20094_0004_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)
rawsce_5 <- read10xCounts(paste0(rdatadir, "20094_0005_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)
rawsce_6 <- read10xCounts(paste0(rdatadir, "20094_0006_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)
rawsce_7 <- read10xCounts(paste0(rdatadir, "20094_0007_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)
rawsce_8 <- read10xCounts(paste0(rdatadir, "20094_0008_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)
rawsce_9 <- read10xCounts(paste0(rdatadir, "20094_0009_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)
rawsce_12 <- read10xCounts(paste0(rdatadir, "20094_0012_A_B/raw_feature_bc_matrix"), type = "sparse", compressed = TRUE)

### Examine and check the data

In [3]:
rawsce_1

class: SingleCellExperiment 
dim: 36604 6794880 
metadata(1): Samples
assays(1): counts
rownames(36604): ENSG00000243485 ENSG00000237613 ... Htag2 Htag3
rowData names(3): ID Symbol Type
colnames: NULL
colData names(2): Sample Barcode
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [4]:
rawsce_list <- c('rawsce_1', 'rawsce_2', 'rawsce_3', 'rawsce_4', 'rawsce_5',
                 'rawsce_6', 'rawsce_7', 'rawsce_8', 'rawsce_9', 'rawsce_12')

In [5]:
for (i in 1:length(rawsce_list)) {
  rawsce <- get(rawsce_list[i])
  br.out <- barcodeRanks(counts(rawsce))
  
  e.out <- emptyDrops(counts(rawsce))  ## Cells that have UMI counts lower than 100 (by defualt) are empty cells.
  print(table(Sig=e.out$FDR <= 0.05, Limited=e.out$Limited))

  is.cell <- e.out$FDR <= 0.05
  print(sum(is.cell, na.rm=TRUE))

  p <- ggplot(data.frame(br.out), aes(x = rank, y = total)) + 
    geom_point() + 
    scale_x_continuous(trans = "log10") +
    scale_y_continuous(trans = "log10") +
    ggtitle(rawsce_list[i]) +
    theme_classic()
	p <- p + 
		geom_hline(aes(yintercept = min(br.out$fitted, na.rm = TRUE),  color = "FDR_0.05"), linetype = "dashed") +
		geom_hline(aes(yintercept = as.numeric(metadata(br.out)$knee), color = "knee"), linetype = "dashed") +
		geom_hline(aes(yintercept = as.numeric(metadata(br.out)$inflection), color = "inflection"), linetype = "dashed") +
		scale_color_manual(name = "Cutoffs",
											values = c("FDR_0.05" = "red",
																	"knee" = "dodgerblue",
																	"inflection"= "forestgreen")) +
		theme(legend.position = "bottom")
  ggsave(filename = paste0(save_path, '/DropletUtils_', rawsce_list[i], '.png'), plot = p, width = 6, height = 6)
  
  colnames(rawsce) = colData(rawsce)$Barcode
  rawsce <- rawsce[,which(e.out$FDR <= 0.05)]
  
  assign(paste0('DropletUtils_', rawsce_list[i]), rawsce)
}

       Limited
Sig     FALSE  TRUE
  FALSE 65019     0
  TRUE   5346  7150
[1] 12496


Warning message in scale_y_continuous(trans = "log10"):
“log-10 transformation introduced infinite values.”


       Limited
Sig     FALSE  TRUE
  FALSE 75152     0
  TRUE   1496  5808
[1] 7304


Warning message in scale_y_continuous(trans = "log10"):
“log-10 transformation introduced infinite values.”


       Limited
Sig     FALSE  TRUE
  FALSE 68606     0
  TRUE    734  5776
[1] 6510


...

Warning message in scale_y_continuous(trans = "log10"):
“log-10 transformation introduced infinite values.”


       Limited
Sig     FALSE  TRUE
  FALSE 67017     0
  TRUE    565  2312
[1] 2877


Warning message in scale_y_continuous(trans = "log10"):
“log-10 transformation introduced infinite values.”


       Limited
Sig     FALSE  TRUE
  FALSE 60520     0
  TRUE    747  6398
[1] 7145


Warning message in scale_y_continuous(trans = "log10"):
“log-10 transformation introduced infinite values.”


### Save the result data

In [6]:
save(DropletUtils_rawsce_1, file = paste0(save_path, '/DropletUtils_filtered_sce_1.RData'))
save(DropletUtils_rawsce_2, file = paste0(save_path, '/DropletUtils_filtered_sce_2.RData'))
save(DropletUtils_rawsce_3, file = paste0(save_path, '/DropletUtils_filtered_sce_3.RData'))
save(DropletUtils_rawsce_4, file = paste0(save_path, '/DropletUtils_filtered_sce_4.RData'))
save(DropletUtils_rawsce_5, file = paste0(save_path, '/DropletUtils_filtered_sce_5.RData'))
save(DropletUtils_rawsce_6, file = paste0(save_path, '/DropletUtils_filtered_sce_6.RData'))
save(DropletUtils_rawsce_7, file = paste0(save_path, '/DropletUtils_filtered_sce_7.RData'))
save(DropletUtils_rawsce_8, file = paste0(save_path, '/DropletUtils_filtered_sce_8.RData'))
save(DropletUtils_rawsce_9, file = paste0(save_path, '/DropletUtils_filtered_sce_9.RData'))
save(DropletUtils_rawsce_12, file = paste0(save_path, '/DropletUtils_filtered_sce_12.RData'))

Reference
Lun, A. T., Riesenfeld, S., Andrews, T., Gomes, T., & Marioni, J. C. (2019). EmptyDrops: distinguishing cells from empty droplets in droplet-based single-cell RNA sequencing data. Genome biology, 20(1), 1-9.

Pekayvaz, K., Leunig, A., Kaiser, R., Joppich, M., Brambs, S., Janjic, A., ... & Nicolai, L. (2022). Protective immune trajectories in early viral containment of non-pneumonic SARS-CoV-2 infection. Nature communications, 13(1), 1-21.

In [7]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 24.04.2 LTS

Matrix products: default
BLAS/LAPACK: /BiO/prog/miniforge3/envs/QC/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Asia/Seoul
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] scran_1.30.0                DropletUtils_1.22.0        
 [3] scater_1.30.1               ggplot2_3.5.2              
 [5] scuttle_1.12.0              SingleCellExperiment_1.24.0
 [7] SummarizedExperiment_1.32.0 Biob